In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 41.2 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 35.8 MB/s 
     |████████████████████████████████| 6.6 MB 37.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install torchinfo

In [ ]:
import os
# from google.colab import drive

import numpy as np
import pandas as pd
import json
import random
import time
import datetime
# from rake_nltk import Rake
import pandas as pd
from scipy import spatial

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from transformers import BertTokenizer, BertModel

import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# df = pd.read_csv('IMDB_Top250Engmovies2_OMDB_Detailed.csv')
df = pd.read_csv('fullset_clean.csv')
df.head()
    
for index, row in df.iterrows():
    df.at[index,'Genre'] = row['Genre'].replace('|',',')

df['Genre_list'] = df['Genre'].map(lambda x: x.split(','))
df['Director_list'] = df['Director'].map(lambda x: x.split(','))
df['Actors_list'] = df['Actors'].map(lambda x: x.split(',')[:3])

df['BoW_genre'] = df['Genre'].map(lambda x: x.lower().replace(',',' '))
df['BoW_director'] = df['Director'].map(lambda x: x.lower().replace(',',' '))
df['BoW_actors'] = df['Actors'].map(lambda x: x.lower().replace(',',' '))

for index, row in df.iterrows():
    df.at[index,'Genre_list'] = [x.lower() for x in row['Genre_list']]
    df.at[index,'Director_list'] = [x.lower() for x in row['Director_list']]
    df.at[index,'Actors_list'] = [x.lower() for x in row['Actors_list']]
    
columns = ['Genre_list', 'Director_list', 'Actors_list']

for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'Bag_of_words_BERT'] = words
    BoW_words = ''
    for col in columns:
        BoW_words += ' '.join(row[col]) + ' '
    df.at[index,'Bag_of_words'] = BoW_words
    genre_director = ''
    for col in columns[:2]:
        genre_director += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'BoW_genre_director_BERT'] = genre_director
    BoW_genre_director = ''
    for col in columns[:2]:
        BoW_genre_director += ' '.join(row[col]) + ' '
    df.at[index,'BoW_genre_director'] = BoW_genre_director
    genre_actors = ''
    for col in columns[0:3:2]:
        genre_actors += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'BoW_genre_actors_BERT'] = genre_actors
    BoW_genre_actors = ''
    for col in columns[0:3:2]:
        BoW_genre_actors += ' '.join(row[col]) + ' '
    df.at[index,'BoW_genre_actors'] = BoW_genre_actors
    director_actors = ''
    for col in columns[1:]:
        director_actors += ' '.join(row[col]) + ' [SEP] '
    df.at[index,'BoW_director_actors_BERT'] = director_actors
    BoW_director_actors = ''
    for col in columns[1:]:
        BoW_director_actors += ' '.join(row[col]) + ' '
    df.at[index,'BoW_director_actors'] = BoW_director_actors
    
# df = df[['Title','Bag_of_words']]
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Director_list,Actors_list,BoW_genre,BoW_director,BoW_actors,Bag_of_words_BERT,Bag_of_words,BoW_genre_director_BERT,BoW_genre_director,BoW_genre_actors_BERT,BoW_genre_actors,BoW_director_actors_BERT,BoW_director_actors
0,0,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...","[adventure, animation, children, comedy, fantasy]",[john lasseter],"[tim allen, tom hanks, don rickles]",adventure animation children comedy fantasy,john lasseter,tim allen tom hanks don rickles jim varney ...,adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy jo...,adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy jo...,adventure animation children comedy fantasy [S...,adventure animation children comedy fantasy ti...,john lasseter [SEP] tim allen tom hanks don ...,john lasseter tim allen tom hanks don rickles
1,1,Jumanji (1995),"Adventure,Children,Fantasy",Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...","[adventure, children, fantasy]",[joe johnston],"[jonathan hyde, bradley pierce, robin williams]",adventure children fantasy,joe johnston,jonathan hyde bradley pierce robin williams ...,adventure children fantasy [SEP] joe johnston ...,adventure children fantasy joe johnston jonath...,adventure children fantasy [SEP] joe johnston ...,adventure children fantasy joe johnston,adventure children fantasy [SEP] jonathan hyde...,adventure children fantasy jonathan hyde brad...,joe johnston [SEP] jonathan hyde bradley pier...,joe johnston jonathan hyde bradley pierce ro...
2,2,Grumpier Old Men (1995),"Comedy,Romance",Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...","[comedy, romance]",[howard deutch],"[jack lemmon, walter matthau, ann-margret ]",comedy romance,howard deutch,jack lemmon walter matthau ann-margret sop...,comedy romance [SEP] howard deutch [SEP] jack ...,comedy romance howard deutch jack lemmon walt...,comedy romance [SEP] howard deutch [SEP],comedy romance howard deutch,comedy romance [SEP] jack lemmon walter matth...,comedy romance jack lemmon walter matthau an...,howard deutch [SEP] jack lemmon walter mattha...,howard deutch jack lemmon walter matthau ann...
3,3,Waiting to Exhale (1995),"Comedy,Drama,Romance",Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...","[comedy, drama, romance]",[forest whitaker],"[angela bassett, loretta devine, whitney hou...",comedy drama romance,forest whitaker,angela bassett loretta devine whitney housto...,comedy drama romance [SEP] forest whitaker [SE...,comedy drama romance forest whitaker angela ba...,comedy drama romance [SEP] forest whitaker [SEP],comedy drama romance forest whitaker,comedy drama romance [SEP] angela bassett lor...,comedy drama romance angela bassett loretta d...,forest whitaker [SEP] angela bassett loretta ...,forest whitaker angela bassett loretta devine...
4,4,Father of the Bride Part II (1995),Comedy,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",[comedy],[charles shyer],"[steve martin, martin short, diane keaton]",comedy,charles shyer,steve martin martin short diane keaton kimb...,comedy [SEP] charles shyer [SEP] steve martin ...,comedy charles shyer steve martin martin shor...,comedy [SEP] charles shyer [SEP],comedy charles shyer,comedy [SEP] steve martin martin short diane...,comedy steve martin martin short diane keaton,charles shyer [SEP] steve martin martin short...,charles shyer steve martin martin short dian...


# Bag of Words search

In [ ]:
# df = pd.read_csv('fullset_BoW.csv')

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])

In [ ]:
# count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
count_matrix_search=count.transform(["crime drama marlon brando"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Director_list,Actors_list,BoW_genre,BoW_director,BoW_actors,Bag_of_words_BERT,Bag_of_words,BoW_genre_director_BERT,BoW_genre_director,BoW_genre_actors_BERT,BoW_genre_actors,BoW_director_actors_BERT,BoW_director_actors
11384,11384,"Fugitive Kind, The (1959)","Crime,Drama,Romance",Sidney Lumet,"Marlon Brando, Anna Magnani, Joanne Woodward, ...","[crime, drama, romance]",[sidney lumet],"[marlon brando, anna magnani, joanne woodward]",crime drama romance,sidney lumet,marlon brando anna magnani joanne woodward ...,crime drama romance [SEP] sidney lumet [SEP] m...,crime drama romance sidney lumet marlon brando...,crime drama romance [SEP] sidney lumet [SEP],crime drama romance sidney lumet,crime drama romance [SEP] marlon brando anna ...,crime drama romance marlon brando anna magnan...,sidney lumet [SEP] marlon brando anna magnani...,sidney lumet marlon brando anna magnani joan...
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]",[francis ford coppola],"[marlon brando, al pacino, james caan]",crime drama,francis ford coppola,marlon brando al pacino james caan richard ...,crime drama [SEP] francis ford coppola [SEP] m...,crime drama francis ford coppola marlon brando...,crime drama [SEP] francis ford coppola [SEP],crime drama francis ford coppola,crime drama [SEP] marlon brando al pacino ja...,crime drama marlon brando al pacino james caan,francis ford coppola [SEP] marlon brando al p...,francis ford coppola marlon brando al pacino ...
1838,1838,On the Waterfront (1954),"Crime,Drama",Elia Kazan,"Marlon Brando, Eva Marie Saint, Lee J. Cobb, K...","[crime, drama]",[elia kazan],"[marlon brando, eva marie saint, lee j. cobb]",crime drama,elia kazan,marlon brando eva marie saint lee j. cobb k...,crime drama [SEP] elia kazan [SEP] marlon bran...,crime drama elia kazan marlon brando eva mari...,crime drama [SEP] elia kazan [SEP],crime drama elia kazan,crime drama [SEP] marlon brando eva marie sai...,crime drama marlon brando eva marie saint le...,elia kazan [SEP] marlon brando eva marie sain...,elia kazan marlon brando eva marie saint lee...
7164,7164,The Chase (1966),"Crime,Drama,Thriller",Arthur Penn,"Marlon Brando,Jane Fonda,Robert Redford,E.G. M...","[crime, drama, thriller]",[arthur penn],"[marlon brando, jane fonda, robert redford]",crime drama thriller,arthur penn,marlon brando jane fonda robert redford e.g. m...,crime drama thriller [SEP] arthur penn [SEP] m...,crime drama thriller arthur penn marlon brando...,crime drama thriller [SEP] arthur penn [SEP],crime drama thriller arthur penn,crime drama thriller [SEP] marlon brando jane ...,crime drama thriller marlon brando jane fonda ...,arthur penn [SEP] marlon brando jane fonda rob...,arthur penn marlon brando jane fonda robert re...
4919,4919,One-Eyed Jacks (1961),Western,Marlon Brando,"Marlon Brando, Karl Malden, Katy Jurado, Pina ...",[western],[marlon brando],"[marlon brando, karl malden, katy jurado]",western,marlon brando,marlon brando karl malden katy jurado pina ...,western [SEP] marlon brando [SEP] marlon brand...,western marlon brando marlon brando karl mald...,western [SEP] marlon brando [SEP],western marlon brando,western [SEP] marlon brando karl malden katy...,western marlon brando karl malden katy jurado,marlon brando [SEP] marlon brando karl malden...,marlon brando marlon brando karl malden katy...
6555,6555,"Ugly American, The (1963)",Drama,George Englund,"Marlon Brando, Eiji Okada, Sandra Church, Pat ...",[drama],[george englund],"[marlon brando, eiji okada, sandra church]",drama,george englund,marlon brando eiji okada sandra church pat ...,drama [SEP] george englund [SEP] marlon brando...,drama george englund marlon brando eiji okada...,drama [SEP] george englund [SEP],drama george englund,drama [SEP] marlon brando eiji okada sandra ...,drama marlon brando eiji okada sandra church,george englund [SEP] marlon brando eiji okada...,ge

# Combination Search

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['BoW_genre_actors'])

In [ ]:
# count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
count_matrix_search=count.transform(["crime drama marlon brando"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Director_list,Actors_list,BoW_genre,BoW_director,BoW_actors,Bag_of_words_BERT,Bag_of_words,BoW_genre_director_BERT,BoW_genre_director,BoW_genre_actors_BERT,BoW_genre_actors,BoW_director_actors_BERT,BoW_director_actors
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]",[francis ford coppola],"[marlon brando, al pacino, james caan]",crime drama,francis ford coppola,marlon brando al pacino james caan richard ...,crime drama [SEP] francis ford coppola [SEP] m...,crime drama francis ford coppola marlon brando...,crime drama [SEP] francis ford coppola [SEP],crime drama francis ford coppola,crime drama [SEP] marlon brando al pacino ja...,crime drama marlon brando al pacino james caan,francis ford coppola [SEP] marlon brando al p...,francis ford coppola marlon brando al pacino ...
7164,7164,The Chase (1966),"Crime,Drama,Thriller",Arthur Penn,"Marlon Brando,Jane Fonda,Robert Redford,E.G. M...","[crime, drama, thriller]",[arthur penn],"[marlon brando, jane fonda, robert redford]",crime drama thriller,arthur penn,marlon brando jane fonda robert redford e.g. m...,crime drama thriller [SEP] arthur penn [SEP] m...,crime drama thriller arthur penn marlon brando...,crime drama thriller [SEP] arthur penn [SEP],crime drama thriller arthur penn,crime drama thriller [SEP] marlon brando jane ...,crime drama thriller marlon brando jane fonda ...,arthur penn [SEP] marlon brando jane fonda rob...,arthur penn marlon brando jane fonda robert re...
1838,1838,On the Waterfront (1954),"Crime,Drama",Elia Kazan,"Marlon Brando, Eva Marie Saint, Lee J. Cobb, K...","[crime, drama]",[elia kazan],"[marlon brando, eva marie saint, lee j. cobb]",crime drama,elia kazan,marlon brando eva marie saint lee j. cobb k...,crime drama [SEP] elia kazan [SEP] marlon bran...,crime drama elia kazan marlon brando eva mari...,crime drama [SEP] elia kazan [SEP],crime drama elia kazan,crime drama [SEP] marlon brando eva marie sai...,crime drama marlon brando eva marie saint le...,elia kazan [SEP] marlon brando eva marie sain...,elia kazan marlon brando eva marie saint lee...
11384,11384,"Fugitive Kind, The (1959)","Crime,Drama,Romance",Sidney Lumet,"Marlon Brando, Anna Magnani, Joanne Woodward, ...","[crime, drama, romance]",[sidney lumet],"[marlon brando, anna magnani, joanne woodward]",crime drama romance,sidney lumet,marlon brando anna magnani joanne woodward ...,crime drama romance [SEP] sidney lumet [SEP] m...,crime drama romance sidney lumet marlon brando...,crime drama romance [SEP] sidney lumet [SEP],crime drama romance sidney lumet,crime drama romance [SEP] marlon brando anna ...,crime drama romance marlon brando anna magnan...,sidney lumet [SEP] marlon brando anna magnani...,sidney lumet marlon brando anna magnani joan...
7879,7879,The Night of the Following Day (1968),"Action,Crime,Drama,Thriller","Hubert Cornfield, Richard Boone","Marlon Brando,Richard Boone,Rita Moreno,Pamela...","[action, crime, drama, thriller]","[hubert cornfield, richard boone]","[marlon brando, richard boone, rita moreno]",action crime drama thriller,hubert cornfield richard boone,marlon brando richard boone rita moreno pamela...,action crime drama thriller [SEP] hubert cornf...,action crime drama thriller hubert cornfield ...,action crime drama thriller [SEP] hubert cornf...,action crime drama thriller hubert cornfield ...,action crime drama thriller [SEP] marlon brand...,action crime drama thriller marlon brando rich...,hubert cornfield richard boone [SEP] marlon b...,hubert cornfield richard boone marlon brando ...
1069,1069,"Streetcar Named Desire, A (1951)",Drama,Elia Kazan,"Marlon Brando, Karl Malden, Kim Hunter, Vivien...",[drama],[elia kazan],"[marlon brando, karl malden, kim hunter]",drama,elia kazan,marlon brando karl malden kim hunter vivien...,drama [SEP] elia kazan [SEP] marlon brando ka...,drama elia kazan marlon brando karl malden k...,drama [SEP] 

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['BoW_genre_director'])

In [ ]:
# count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
count_matrix_search=count.transform(["crime drama francis ford coppala"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Director_list,Actors_list,BoW_genre,BoW_director,BoW_actors,Bag_of_words_BERT,Bag_of_words,BoW_genre_director_BERT,BoW_genre_director,BoW_genre_actors_BERT,BoW_genre_actors,BoW_director_actors_BERT,BoW_director_actors
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]",[francis ford coppola],"[marlon brando, al pacino, james caan]",crime drama,francis ford coppola,marlon brando al pacino james caan richard ...,crime drama [SEP] francis ford coppola [SEP] m...,crime drama francis ford coppola marlon brando...,crime drama [SEP] francis ford coppola [SEP],crime drama francis ford coppola,crime drama [SEP] marlon brando al pacino ja...,crime drama marlon brando al pacino james caan,francis ford coppola [SEP] marlon brando al p...,francis ford coppola marlon brando al pacino ...
1178,1178,"Godfather: Part II, The (1974)","Crime,Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...","[crime, drama]",[francis ford coppola],"[al pacino, robert duvall, diane keaton]",crime drama,francis ford coppola,al pacino robert duvall diane keaton robert...,crime drama [SEP] francis ford coppola [SEP] a...,crime drama francis ford coppola al pacino ro...,crime drama [SEP] francis ford coppola [SEP],crime drama francis ford coppola,crime drama [SEP] al pacino robert duvall di...,crime drama al pacino robert duvall diane ke...,francis ford coppola [SEP] al pacino robert d...,francis ford coppola al pacino robert duvall ...
1916,1916,"Godfather: Part III, The (1990)","Crime,Drama,Mystery,Thriller",Francis Ford Coppola,"Al Pacino, Diane Keaton, Talia Shire, Andy Gar...","[crime, drama, mystery, thriller]",[francis ford coppola],"[al pacino, diane keaton, talia shire]",crime drama mystery thriller,francis ford coppola,al pacino diane keaton talia shire andy gar...,crime drama mystery thriller [SEP] francis for...,crime drama mystery thriller francis ford copp...,crime drama mystery thriller [SEP] francis for...,crime drama mystery thriller francis ford copp...,crime drama mystery thriller [SEP] al pacino ...,crime drama mystery thriller al pacino diane ...,francis ford coppola [SEP] al pacino diane ke...,francis ford coppola al pacino diane keaton ...
2006,2006,"Outsiders, The (1983)",Drama,Francis Ford Coppola,"Matt Dillon, C. Thomas Howell, Ralph Macchio, ...",[drama],[francis ford coppola],"[matt dillon, c. thomas howell, ralph macchio]",drama,francis ford coppola,matt dillon c. thomas howell ralph macchio ...,drama [SEP] francis ford coppola [SEP] matt di...,drama francis ford coppola matt dillon c. tho...,drama [SEP] francis ford coppola [SEP],drama francis ford coppola,drama [SEP] matt dillon c. thomas howell ral...,drama matt dillon c. thomas howell ralph mac...,francis ford coppola [SEP] matt dillon c. tho...,francis ford coppola matt dillon c. thomas ho...
14739,14739,"Rain People, The (1969)",Drama,Francis Ford Coppola,"James Caan, Shirley Knight, Robert Duvall",[drama],[francis ford coppola],"[james caan, shirley knight, robert duvall]",drama,francis ford coppola,james caan shirley knight robert duvall,drama [SEP] francis ford coppola [SEP] james c...,drama francis ford coppola james caan shirley...,drama [SEP] francis ford coppola [SEP],drama francis ford coppola,drama [SEP] james caan shirley knight robert...,drama james caan shirley knight robert duvall,francis ford coppola [SEP] james caan shirley...,francis ford coppola james caan shirley knigh...
22831,22831,Enter Arsene Lupin (1944),"Crime,Drama",Ford Beebe,"Charles Korvin,Ella Raines,J. Carrol Naish,Geo...","[crime, drama]",[ford beebe],"[charles korvin, ella raines, j. carrol naish]",crime drama,ford beebe,charles korvin ella raines j. carrol naish geo...,crime drama [SEP] ford beebe [SEP] charles kor...,crime drama ford beebe charles korvin ella rai...,crime drama [SEP] ford beebe [SEP],crime drama ford beebe,crime drama [SE

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['BoW_director_actors'])

In [ ]:
# count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
count_matrix_search=count.transform(["francis ford coppala marlon brando"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Director_list,Actors_list,BoW_genre,BoW_director,BoW_actors,Bag_of_words_BERT,Bag_of_words,BoW_genre_director_BERT,BoW_genre_director,BoW_genre_actors_BERT,BoW_genre_actors,BoW_director_actors_BERT,BoW_director_actors
1165,1165,Apocalypse Now (1979),"Action,Drama,War",Francis Ford Coppola,"Marlon Brando, Robert Duvall, Frederic Forrest...","[action, drama, war]",[francis ford coppola],"[marlon brando, robert duvall, frederic forr...",action drama war,francis ford coppola,marlon brando robert duvall frederic forrest...,action drama war [SEP] francis ford coppola [S...,action drama war francis ford coppola marlon b...,action drama war [SEP] francis ford coppola [S...,action drama war francis ford coppola,action drama war [SEP] marlon brando robert d...,action drama war marlon brando robert duvall ...,francis ford coppola [SEP] marlon brando robe...,francis ford coppola marlon brando robert duv...
46856,46856,The Godfather Family: A Look Inside (1990),"Action,Documentary,Drama",Jeff Werner,"Francis Ford Coppola,Mario Puzo,Marlon Brando,...","[action, documentary, drama]",[jeff werner],"[francis ford coppola, mario puzo, marlon brando]",action documentary drama,jeff werner,francis ford coppola mario puzo marlon brando ...,action documentary drama [SEP] jeff werner [SE...,action documentary drama jeff werner francis f...,action documentary drama [SEP] jeff werner [SEP],action documentary drama jeff werner,action documentary drama [SEP] francis ford co...,action documentary drama francis ford coppola ...,jeff werner [SEP] francis ford coppola mario p...,jeff werner francis ford coppola mario puzo ma...
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]",[francis ford coppola],"[marlon brando, al pacino, james caan]",crime drama,francis ford coppola,marlon brando al pacino james caan richard ...,crime drama [SEP] francis ford coppola [SEP] m...,crime drama francis ford coppola marlon brando...,crime drama [SEP] francis ford coppola [SEP],crime drama francis ford coppola,crime drama [SEP] marlon brando al pacino ja...,crime drama marlon brando al pacino james caan,francis ford coppola [SEP] marlon brando al p...,francis ford coppola marlon brando al pacino ...
4919,4919,One-Eyed Jacks (1961),Western,Marlon Brando,"Marlon Brando, Karl Malden, Katy Jurado, Pina ...",[western],[marlon brando],"[marlon brando, karl malden, katy jurado]",western,marlon brando,marlon brando karl malden katy jurado pina ...,western [SEP] marlon brando [SEP] marlon brand...,western marlon brando marlon brando karl mald...,western [SEP] marlon brando [SEP],western marlon brando,western [SEP] marlon brando karl malden katy...,western marlon brando karl malden katy jurado,marlon brando [SEP] marlon brando karl malden...,marlon brando marlon brando karl malden katy...
10089,10089,"Teahouse of the August Moon, The (1956)",Comedy,Daniel Mann,"Marlon Brando, Glenn Ford, Machiko Kyô, Eddie ...",[comedy],[daniel mann],"[marlon brando, glenn ford, machiko kyô]",comedy,daniel mann,marlon brando glenn ford machiko kyô eddie ...,comedy [SEP] daniel mann [SEP] marlon brando ...,comedy daniel mann marlon brando glenn ford ...,comedy [SEP] daniel mann [SEP],comedy daniel mann,comedy [SEP] marlon brando glenn ford machik...,comedy marlon brando glenn ford machiko kyô,daniel mann [SEP] marlon brando glenn ford m...,daniel mann marlon brando glenn ford machiko...
19101,19101,Fog City Mavericks (2007),Documentary,Gary Leyva,"George Lucas, Francis Ford Coppola",[documentary],[gary leyva],"[george lucas, francis ford coppola]",documentary,gary leyva,george lucas francis ford coppola,documentary [SEP] gary leyva [SEP] george luca...,documentary gary leyva george lucas francis f...,documentary [SEP] gary leyva [SEP],documentary gary leyva,documentary [SEP] george lucas francis ford c...,documentary george lucas francis ford coppola,gary leyva [SEP] g

# Tf Idf Search

In [ ]:
count = TfidfVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])

In [ ]:
# count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
count_matrix_search=count.transform(["crime drama marlon brando"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

,Unnamed: 0,Title,Genre,Director,Actors,Genre_list,Director_list,Actors_list,BoW_genre,BoW_director,BoW_actors,Bag_of_words_BERT,Bag_of_words,BoW_genre_director_BERT,BoW_genre_director,BoW_genre_actors_BERT,BoW_genre_actors,BoW_director_actors_BERT,BoW_director_actors
4919,4919,One-Eyed Jacks (1961),Western,Marlon Brando,"Marlon Brando, Karl Malden, Katy Jurado, Pina ...",[western],[marlon brando],"[marlon brando, karl malden, katy jurado]",western,marlon brando,marlon brando karl malden katy jurado pina ...,western [SEP] marlon brando [SEP] marlon brand...,western marlon brando marlon brando karl mald...,western [SEP] marlon brando [SEP],western marlon brando,western [SEP] marlon brando karl malden katy...,western marlon brando karl malden katy jurado,marlon brando [SEP] marlon brando karl malden...,marlon brando marlon brando karl malden katy...
7498,7498,Reflections in a Golden Eye (1967),Drama,John Huston,"Elizabeth Taylor, Marlon Brando, Brian Keith, ...",[drama],[john huston],"[elizabeth taylor, marlon brando, brian keith]",drama,john huston,elizabeth taylor marlon brando brian keith ...,drama [SEP] john huston [SEP] elizabeth taylor...,drama john huston elizabeth taylor marlon bra...,drama [SEP] john huston [SEP],drama john huston,drama [SEP] elizabeth taylor marlon brando b...,drama elizabeth taylor marlon brando brian k...,john huston [SEP] elizabeth taylor marlon bra...,john huston elizabeth taylor marlon brando b...
7164,7164,The Chase (1966),"Crime,Drama,Thriller",Arthur Penn,"Marlon Brando,Jane Fonda,Robert Redford,E.G. M...","[crime, drama, thriller]",[arthur penn],"[marlon brando, jane fonda, robert redford]",crime drama thriller,arthur penn,marlon brando jane fonda robert redford e.g. m...,crime drama thriller [SEP] arthur penn [SEP] m...,crime drama thriller arthur penn marlon brando...,crime drama thriller [SEP] arthur penn [SEP],crime drama thriller arthur penn,crime drama thriller [SEP] marlon brando jane ...,crime drama thriller marlon brando jane fonda ...,arthur penn [SEP] marlon brando jane fonda rob...,arthur penn marlon brando jane fonda robert re...
6798,6798,"Freshman, The (1990)","Comedy,Crime",Andrew Bergman,"Matthew Broderick, Marlon Brando, Bruno Kirby,...","[comedy, crime]",[andrew bergman],"[matthew broderick, marlon brando, bruno kirby]",comedy crime,andrew bergman,matthew broderick marlon brando bruno kirby ...,comedy crime [SEP] andrew bergman [SEP] matthe...,comedy crime andrew bergman matthew broderick ...,comedy crime [SEP] andrew bergman [SEP],comedy crime andrew bergman,comedy crime [SEP] matthew broderick marlon b...,comedy crime matthew broderick marlon brando ...,andrew bergman [SEP] matthew broderick marlon...,andrew bergman matthew broderick marlon brand...
11610,11610,Bedtime Story (1964),Comedy,Ralph Levy,"Marlon Brando, David Niven, Shirley Jones",[comedy],[ralph levy],"[marlon brando, david niven, shirley jones ]",comedy,ralph levy,marlon brando david niven shirley jones,comedy [SEP] ralph levy [SEP] marlon brando d...,comedy ralph levy marlon brando david niven ...,comedy [SEP] ralph levy [SEP],comedy ralph levy,comedy [SEP] marlon brando david niven shirl...,comedy marlon brando david niven shirley jon...,ralph levy [SEP] marlon brando david niven s...,ralph levy marlon brando david niven shirley...
834,834,"Godfather, The (1972)","Crime,Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...","[crime, drama]",[francis ford coppola],"[marlon brando, al pacino, james caan]",crime drama,francis ford coppola,marlon brando al pacino james caan richard ...,crime drama [SEP] francis ford coppola [SEP] m...,crime drama francis ford coppola marlon brando...,crime drama [SEP] francis ford coppola [SEP],crime drama francis ford coppola,crime drama [SEP] marlon brando al pacino ja...,crime drama marlon brando al pacino james caan,francis ford coppola [SEP] marlon brando al p...,francis ford coppola marlon brando al pacino ...
4899,4899,"Formula, The (1980)",Thril

# Testing random masking

In [ ]:
import random

correct=0

# for i in range(len(df)):
for i in range(5000):
  target=df.iloc[i].Title
  gen=df.iloc[i].Genre_list.copy()
  dir=df.iloc[i].Director_list.copy()
  acts=df.iloc[i].Actors_list.copy()

  full_list = [gen,dir,acts]

  # print(full_list)
  # rem=random.randint(0,4)
  # print(rem)
  
  for _ in range(4):
    category=random.randint(0,len(full_list)-1)

    word=random.randint(0,len(full_list[category])-1)

    full_list[category][word]=""
  # print(full_list)

  search_words = ''
  for ind in range(len(full_list)):
      search_words += ' '.join(full_list[ind]) + ' [SEP] '
      # print(search_words)

  # count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
  count_matrix_search=count.transform([search_words])
  # count_matrix_search=count.transform(["blah boee beep"])
  cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

  def recommend_search(cosine_sim = cosine_sim):
      recommended_movies = []
      score_series = pd.Series(cosine_sim).sort_values(ascending = False)
      top_10_indices = list(score_series.iloc[:10].index)
      
      return df.iloc[top_10_indices]

  list_of_movies=recommend_search(cosine_sim.flatten())

  if target in list(list_of_movies.Title):
    correct+=1
print(correct)

4783


In [ ]:
import random

correct=0

# for i in range(len(df)):
for i in range(5000):
  target=df.iloc[i].Title
  gen=df.iloc[i].Genre_list.copy()
  dir=df.iloc[i].Director_list.copy()
  acts=df.iloc[i].Actors_list.copy()

  full_list = [gen,[""],acts]

  # print(full_list)
  # rem=random.randint(0,4)
  # print(rem)
  
  # for _ in range(2):
  #   category=random.randint(0,len(full_list)-1)

  #   word=random.randint(0,len(full_list[category])-1)

  #   full_list[category][word]=""
  # print(full_list)

  search_words = ''
  for ind in range(len(full_list)):
      search_words += ' '.join(full_list[ind]) + ' [SEP] '
      # print(search_words)


  count = TfidfVectorizer()
  count_matrix = count.fit_transform(df['BoW_genre_actors'])
  # count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
  count_matrix_search=count.transform([search_words])
  # count_matrix_search=count.transform(["blah boee beep"])
  cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

  def recommend_search(cosine_sim = cosine_sim):
      recommended_movies = []
      score_series = pd.Series(cosine_sim).sort_values(ascending = False)
      top_10_indices = list(score_series.iloc[:10].index)
      
      return df.iloc[top_10_indices]

  list_of_movies=recommend_search(cosine_sim.flatten())

  if target in list(list_of_movies.Title):
    correct+=1
print(correct)

5000


In [ ]:
import random

correct=0

# for i in range(len(df)):
for i in range(5000):
  target=df.iloc[i].Title
  gen=df.iloc[i].Genre_list.copy()
  dir=df.iloc[i].Director_list.copy()
  acts=df.iloc[i].Actors_list.copy()

  full_list = [[""],dir,acts]

  # print(full_list)
  # rem=random.randint(0,4)
  # print(rem)
  
  # for _ in range(2):
  #   category=random.randint(0,len(full_list)-1)

  #   word=random.randint(0,len(full_list[category])-1)

  #   full_list[category][word]=""
  # print(full_list)

  search_words = ''
  for ind in range(len(full_list)):
      search_words += ' '.join(full_list[ind]) + ' [SEP] '
      # print(search_words)

  count = TfidfVectorizer()
  count_matrix = count.fit_transform(df['BoW_director_actors'])
  # count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
  count_matrix_search=count.transform([search_words])
  # count_matrix_search=count.transform(["blah boee beep"])
  cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

  def recommend_search(cosine_sim = cosine_sim):
      recommended_movies = []
      score_series = pd.Series(cosine_sim).sort_values(ascending = False)
      top_10_indices = list(score_series.iloc[:10].index)
      
      return df.iloc[top_10_indices]

  list_of_movies=recommend_search(cosine_sim.flatten())

  if target in list(list_of_movies.Title):
    correct+=1
print(correct)

In [ ]:
import random

correct=0

# for i in range(len(df)):
for i in range(5000):
  target=df.iloc[i].Title
  gen=df.iloc[i].Genre_list.copy()
  dir=df.iloc[i].Director_list.copy()
  acts=df.iloc[i].Actors_list.copy()

  full_list = [gen,dir,[""]]

  # print(full_list)
  # rem=random.randint(0,4)
  # print(rem)
  
  # for _ in range(2):
  #   category=random.randint(0,len(full_list)-1)

  #   word=random.randint(0,len(full_list[category])-1)

  #   full_list[category][word]=""
  # print(full_list)

  search_words = ''
  for ind in range(len(full_list)):
      search_words += ' '.join(full_list[ind]) + ' [SEP] '
      # print(search_words)

    count = TfidfVectorizer()
  count_matrix = count.fit_transform(df['BoW_genre_director'])
  # count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
  count_matrix_search=count.transform([search_words])
  # count_matrix_search=count.transform(["blah boee beep"])
  cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

  def recommend_search(cosine_sim = cosine_sim):
      recommended_movies = []
      score_series = pd.Series(cosine_sim).sort_values(ascending = False)
      top_10_indices = list(score_series.iloc[:10].index)
      
      return df.iloc[top_10_indices]

  list_of_movies=recommend_search(cosine_sim.flatten())

  if target in list(list_of_movies.Title):
    correct+=1
print(correct)

In [ ]:
count_matrix_search=count.transform(["romance thea sharrock emelia clarke ]"])
# count_matrix_search=count.transform(["blah boee beep"])
cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

def recommend_search(cosine_sim = cosine_sim):
    recommended_movies = []
    score_series = pd.Series(cosine_sim).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[:10].index)
    
    return df.iloc[top_10_indices]

recommend_search(cosine_sim.flatten())

# Generate BERT embeddings

In [ ]:
# Print Torch version
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print('-' * 10)

# Check GPU
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
seed=17

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.cuda.empty_cache()



In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
text = list(df['Bag_of_words_BERT'])

In [ ]:
# Pytorch Dataset - Encodes text with BERT Tokenizer

class Text_Dataset(torch.utils.data.Dataset):
    def __init__(self, text_data, process_length):
        self.text = [tokenizer(text, padding='max_length', 
                              max_length=process_length, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')
                            for text in text_data]

    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        batch_text = self.text[idx]
        
        return batch_text
        

In [ ]:
text_length = 16
batch_size = 64

text_dataset = Text_Dataset(text, text_length)

text_loader = torch.utils.data.DataLoader(text_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
text_dataset.text[0]

In [ ]:
# Define Model
class BERT_embed(nn.Module):

    def __init__(self):
        super(BERT_embed, self).__init__()

        options_name = "bert-base-uncased"
        self.encoder = BertModel.from_pretrained(options_name, output_attentions=False, output_hidden_states=True)

        
    def forward(self, batch_ids, batch_mask):
        output = self.encoder(input_ids=batch_ids, attention_mask=batch_mask)

        return output

In [ ]:
testmodel = BERT_embed()
summary(testmodel)

In [ ]:
# Train Function

def train_model(model, dataloaders, num_epochs=1):
    
    model.eval()
    
    est = False
    with torch.no_grad():

      # iterate over dataloader
      for inputs in dataloaders['val']:
          # Send to GPU
          inputs_ids = inputs['input_ids'].squeeze().to(device)
          inputs_mask = inputs['attention_mask'].squeeze().to(device)
          # print(inputs_ids.size())
          # print(labels)
          
          # Fit to model
          outputs = model(inputs_ids, inputs_mask)
          # print(outputs[0].size())

          if est == False:
            # res1=outputs[0]
            res2=outputs[1].detach().cpu()
            # res3=outputs[2]
            est = True
          else:
            # res1 = torch.cat((res1,outputs[0]),0)
            res2 = torch.cat((res2,outputs[1].detach().cpu()),0)
            # res3 = torch.cat((res3,outputs[2]),0)
          # print(res2.size())
              
    return model, outputs, res2

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':text_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['BoW_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'BoW_embedding'] = res2.detach().numpy()[index]

In [ ]:
import random

correct=0

for i in range(1000):
  target=df.iloc[i].Title
  gen=df.iloc[i].Genre_list.copy()
  dir=df.iloc[i].Director_list.copy()
  acts=df.iloc[i].Actors_list.copy()

  full_list = [gen,dir,acts]

  # print(full_list)
  # rem=random.randint(0,4)
  # print(rem)
  
  for _ in range(1):
    category=random.randint(0,len(full_list)-1)

    word=random.randint(0,len(full_list[category])-1)

    full_list[category][word]=""
  # print(full_list)

  search_words = ''
  for ind in range(len(full_list)):
      search_words += ' '.join(full_list[ind]) + ' [SEP] '
      # print(search_words)

  search_tok=tokenizer(search_words, padding='max_length', 
                              max_length=16, 
                              truncation=True, 
                              return_attention_mask = True, 
                              return_tensors = 'pt')

  search_out = model(search_tok['input_ids'].to(device), search_tok['attention_mask'].to(device))

  cosine_sim_search = cosine_similarity(np.vstack(df.BoW_embedding), search_out[1].detach().cpu().numpy())

  def recommend_search(cosine_sim = cosine_sim):
      recommended_movies = []
      score_series = pd.Series(cosine_sim).sort_values(ascending = False)
      top_10_indices = list(score_series.iloc[:10].index)
      
      return df.iloc[top_10_indices]

  list_of_movies=recommend_search(cosine_sim_search.flatten())

  if target in list(list_of_movies.Title):
    correct+=1
print(correct)

# BERT embeddings for individual categories

In [ ]:
text_length = 16
batch_size = 64

genre_dataset = Text_Dataset(list(df['Genre']), text_length)

genre_loader = torch.utils.data.DataLoader(genre_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':genre_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['genre_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'genre_embedding'] = res2.detach().numpy()[index]

In [ ]:
text_length = 16
batch_size = 64

director_dataset = Text_Dataset(list(df['Director']), text_length)

director_loader = torch.utils.data.DataLoader(director_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':director_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['director_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'director_embedding'] = res2.detach().numpy()[index]

In [ ]:
text_length = 16
batch_size = 64

actors_dataset = Text_Dataset(list(df['Actors']), text_length)

actors_loader = torch.utils.data.DataLoader(actors_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':actors_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['actors_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'actors_embedding'] = res2.detach().numpy()[index]

In [ ]:
text_length = 16
batch_size = 64

director_actors_dataset = Text_Dataset(list(df['BoW_director_actors']), text_length)

director_actors_loader = torch.utils.data.DataLoader(director_actors_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':director_actors_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['director_actors_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'director_actors_embedding'] = res2.detach().numpy()[index]

In [ ]:
text_length = 16
batch_size = 64

genre_actors_dataset = Text_Dataset(list(df['BoW_genre_actors']), text_length)

genre_actors_loader = torch.utils.data.DataLoader(genre_actors_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':genre_actors_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['genre_actors_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'genre_actors_embedding'] = res2.detach().numpy()[index]

In [ ]:
text_length = 16
batch_size = 64

genre_director_dataset = Text_Dataset(list(df['BoW_genre_director']), text_length)

genre_director_loader = torch.utils.data.DataLoader(genre_director_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_name = 'BERT_Intent'
model = BERT_embed()
model.to(device)

dataloaders_dict = {'val':genre_director_loader}
model, outputs, res2 = train_model(model, dataloaders_dict)

In [ ]:
df['genre_director_embedding'] = ''

In [ ]:
for index, row in df.iterrows():
    df.at[index,'genre_director_embedding'] = res2.detach().numpy()[index]

In [ ]:
import random

correct=0

for i in range(1000):
  target=df.iloc[i].Title
  gen=df.iloc[i].Genre_list.copy()
  dir=df.iloc[i].Director_list.copy()
  acts=df.iloc[i].Actors_list.copy()

  full_list = [gen,dir,acts]

  # print(full_list)
  # rem=random.randint(0,4)
  # print(rem)
  
  for _ in range(4):
    category=random.randint(0,len(full_list)-1)

    word=random.randint(0,len(full_list[category])-1)

    full_list[category][word]=""
  # print(full_list)

  search_words = ''
  for ind in range(len(full_list)):
      search_words += ' '.join(full_list[ind]) + ' [SEP] '
      # print(search_words)

  # count_matrix_search=count.transform(["crime drama [SEP] francis ford coppala"])
  count_matrix_search=count.transform([search_words])
  # count_matrix_search=count.transform(["blah boee beep"])
  cosine_sim = cosine_similarity(count_matrix, count_matrix_search)

  def recommend_search(cosine_sim = cosine_sim):
      recommended_movies = []
      score_series = pd.Series(cosine_sim).sort_values(ascending = False)
      top_10_indices = list(score_series.iloc[:10].index)
      
      return df.iloc[top_10_indices]

  list_of_movies=recommend_search(cosine_sim.flatten())

  if target in list(list_of_movies.Title):
    correct+=1
print(correct)